**Aim**: To get pretrained imagenet features generated by Xception on the dataset and train a softmax layer for classification. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from tqdm import  tqdm
print(os.listdir("../input"))
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,GlobalAvgPool2D
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import Xception
from keras.applications import InceptionV3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import time

In [ ]:
df = pd.read_csv("../input/labels.csv")
df.info()

In [ ]:
# sam = pd.read_csv("../input/sample_submission.csv")
# sam.head(5)

In [ ]:
breed_ls = list(df.groupby('breed').count().sort_values(by='id', ascending=False).index)

In [ ]:
import random
import cv2
from keras.preprocessing import image

def getRandomImageList(breed_name, no_of_samples=60):
    global df
    random_images = []
    for index, row in df.iterrows():
        if row['breed'] == breed_name:
            random_images.append(row['id'])
    random_images = random.sample(random_images, no_of_samples)
    return random_images

def readImgResize(name,path,dim=150):
    img = cv2.imread(path+name)
    img = cv2.resize(img,(dim,dim))
    return image.img_to_array(img)

In [ ]:
INPUT_SIZE = 150
num_class=NUM_CLASSES=120
samples = 65


image_label = []
num = 0
import tqdm
for i,breed in tqdm.tqdm( enumerate(breed_ls[:num_class])):

    ls = getRandomImageList(breed,samples) 
    image_label.extend(ls)
    

In [ ]:
new_df = pd.DataFrame({"id":image_label})
new_df = pd.merge(new_df, df, how='inner', on=['id'])

In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat = le.fit_transform(new_df.breed)
from keras.utils.np_utils import to_categorical
mat = to_categorical(cat)

training_data = np.zeros(shape=(len(new_df.id),INPUT_SIZE,INPUT_SIZE,3))

for i,j in tqdm.tqdm(enumerate(new_df["id"])):
    training_data[i]=readImgResize(j+".jpg",path="../input/train/")
    



In [ ]:
# from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,GlobalAvgPool2D
from keras.layers import Conv2D, MaxPooling2D
import keras
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( training_data, mat, test_size=0.05, random_state=11)
del training_data, mat, new_df, df

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,7))
for i in range(6):
    idx = random.randint(0,len(X_train))
    name = le.inverse_transform(y_train[idx].argmax())
    plt.subplot(2,3,i+1)
    plt.imshow(X_train[idx]/255)
    plt.xlabel(str(name))


In [ ]:
from keras.models import Model
from keras.optimizers import adam

def getModel(model_name,epochs=5):
    im_size = 150
    global X_train
    global y_train
    
    global X_test
    global y_test
    
    base_model = model_name(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we wi`l train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(adam(lr=0.000001),loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    model.fit(X_train/255, y_train, epochs=epochs, validation_data=(X_test/255, y_test), verbose=1)
    
    return(model)

In [ ]:
xcep = getModel(Xception,epochs=60)

The model seems to be working fine. It requires a bit more training before it starts overfitting and as of now the losses of training and validation data are of similar magnititude. Hope to see how this network performs when we use image augmentation and around 150 epochs. 

In [ ]:
del X_train,X_test,y_train,y_test

In [ ]:
te = os.listdir("../input/test/")
te_in = np.zeros((len(te),150,150,3))
for num , i in enumerate(te):
    img = readImgResize(i,path="../input/test/")/255
    te_in[num]=img
    

In [ ]:
pred = xcep.predict(te_in)
submission = pd.DataFrame(pred , columns =le.classes_.tolist())
submission["id"]=[i.split(".")[0] for i in os.listdir("../input/test/")]
submission = submission[["id"]+submission.columns[:-1].tolist()]
submission.sort_values(['id'])
submission.to_csv("submission.csv",index = False)